In [20]:
# import Pandas
import pandas as pd

# import os
import os

In [109]:
# Input data
# 1) NBA Player Stats since 1950
# https://www.kaggle.com/drgilermo/nba-players-stats
# Flat file (CSV) dataset from Kaggle
player_stats_inp = pd.read_csv(os.path.join("..", "RawData", "Seasons_Stats.csv"))

# 2) NBA Play Salaries 1999-2019
# http://www.espn.com/nba/salaries
# Structured dataset scrapped from web
# See PreScripts/web_scrape_espn.ipynb for the scraping process
player_salary_inp = pd.read_csv(os.path.join("..", "RawData", "player_salary.csv"))

# 3) NBA Salary Cap by History
# https://www.basketball-reference.com/contracts/salary-cap-history.html
# SQL dataset (originally CSV, exported to SQL externally using Python Alchemy)
salary_cap_inp = pd.read_csv(os.path.join("..", "CleanData", "salary_cap_clean.csv"))

In [63]:
# Clean up dataset 1 to get ready for merging
# keep useful columns
player_stats = player_stats_inp[["Year", "Player", "Age", "FG%", "eFG%", "FT%", "PTS"]]

# Get rid of NaN values
player_stats = player_stats.dropna()

# Make the values that should be integer integer
player_stats.Year = player_stats.Year.astype(int)
player_stats.Age = player_stats.Age.astype(int)

# rname column
player_stats = player_stats.rename(columns={"Player": "Name"})

player_stats.head()

,Year,Name,Age,FG%,eFG%,FT%,PTS
0,1950,Curly Armstrong,31,0.279,0.279,0.705,458.0
1,1950,Cliff Barker,29,0.372,0.372,0.708,279.0
2,1950,Leo Barnhorst,25,0.349,0.349,0.698,438.0
3,1950,Ed Bartels,24,0.256,0.256,0.559,63.0
4,1950,Ed Bartels,24,0.256,0.256,0.548,59.0


In [26]:
# Clean up dataset 2 to get ready for merging
player_salary = player_salary_inp[["Name", "Position", "Team", "Salary", "Year"]]


player_salary.tail()

,Name,Position,Team,Salary,Year
7505,Al Harrington,PF,Indiana Pacers,"$745,000",2000
7506,Tim James,F,Miami Heat,"$728,000",2000
7507,Jumaine Jones,SF,Philadelphia 76ers,"$684,000",2000
7508,Scott Padgett,PF,Utah Jazz,"$679,000",2000
7509,Rafer Alston,PG,Milwaukee Bucks,"$301,000",2000


In [153]:
# Clean up salary cap data
salary_cap = salary_cap_inp.rename(columns={"SalaryCap": "Salary Cap"})
salary_cap["Salary Cap"] = salary_cap["Salary Cap"].astype(int)

salary_2018 = salary_cap[salary_cap["Year"] == 2018]["Salary Cap"]

# Get the value of most recent year
salary_2018 = salary_2018.values[0]


In [106]:
# Inner merge datasets 1 and 2
player_stats_salary = player_stats.merge(player_salary, how = "inner", on = ["Name", "Year"])
player_stats_salary.head()

,Year,Name,Age,FG%,eFG%,FT%,PTS,Position,Team,Salary
0,2000,Ray Allen,24,0.455,0.516,0.887,1809.0,SG,Milwaukee Bucks,"$9,000,000"
1,2000,Rafer Alston,23,0.284,0.300,0.750,60.0,PG,Milwaukee Bucks,"$301,000"
2,2000,Kenny Anderson,29,0.440,0.483,0.775,1149.0,PG,Boston Celtics,"$6,680,000"
3,2000,Darrell Armstrong,31,0.433,0.494,0.911,1330.0,PG,Orlando Magic,"$4,125,000"
4,2000,Isaac Austin,30,0.429,0.430,0.686,397.0,C,Washington Wizards,"$4,819,000"


In [107]:
# Convert Salary to numbers
player_stats_salary["Salary"] = player_stats_salary["Salary"].str.strip("$")
player_stats_salary["Salary"] = player_stats_salary["Salary"].str.replace(',', '').astype(int)
player_stats_salary.head()

,Year,Name,Age,FG%,eFG%,FT%,PTS,Position,Team,Salary
0,2000,Ray Allen,24,0.455,0.516,0.887,1809.0,SG,Milwaukee Bucks,9000000
1,2000,Rafer Alston,23,0.284,0.300,0.750,60.0,PG,Milwaukee Bucks,301000
2,2000,Kenny Anderson,29,0.440,0.483,0.775,1149.0,PG,Boston Celtics,6680000
3,2000,Darrell Armstrong,31,0.433,0.494,0.911,1330.0,PG,Orlando Magic,4125000
4,2000,Isaac Austin,30,0.429,0.430,0.686,397.0,C,Washington Wizards,4819000


In [155]:
# Normalize with salary cap by the year
player_normalized = player_stats_salary.merge(salary_cap, how = "inner", on = "Year")

player_normalized["Salary Normalized"] = player_normalized["Salary"] / player_normalized["Salary Cap"] * salary_2018

player_normalized.tail()

,Year,Name,Age,FG%,eFG%,FT%,PTS,Position,Team,Salary,Salary Normalized,Salary Cap
7088,2017,Cody Zeller,24,0.571,0.571,0.679,639.0,C,Charlotte Hornets,5318313,5.597948e+06,94143000
7089,2017,Tyler Zeller,27,0.494,0.494,0.564,178.0,C,Boston Celtics,8000000,8.420637e+06,94143000
7090,2017,Stephen Zimmerman,20,0.323,0.323,0.600,23.0,C,Orlando Magic,950000,9.999506e+05,94143000
7091,2017,Paul Zipser,22,0.398,0.473,0.775,240.0,F,Chicago Bulls,750000,7.894347e+05,94143000
7092,2017,Ivica Zubac,19,0.529,0.529,0.653,284.0,C,Los Angeles Lakers,1034956,1.089374e+06,94143000


In [156]:
# analyze position info
# The five basketball positions normally employed by organized basketball teams are 
# the point guard (PG), the shooting guard (SG), the small forward (SF), the power forward (PF), and the center (C).
player_gby = player_normalized.groupby("Position")
player_gby.describe()

Age                                                         FG%  \
           count       mean       std   min   25%   50%    75%   max   count   
Position                                                                       
 C        1370.0  26.608759  3.981357  18.0  24.0  26.0  30.00  40.0  1370.0   
 F         324.0  27.040123  4.395494  20.0  24.0  26.0  30.25  38.0   324.0   
 G         351.0  27.150997  4.760251  19.0  23.0  26.0  31.00  39.0   351.0   
 GF          1.0  26.000000       NaN  26.0  26.0  26.0  26.00  26.0     1.0   
 PF       1359.0  27.084621  4.141395  19.0  24.0  27.0  30.00  40.0  1359.0   
 PG       1309.0  27.000000  4.238494  19.0  24.0  26.0  30.00  39.0  1309.0   
 SF       1136.0  26.433979  4.091363  18.0  23.0  26.0  29.00  40.0  1136.0   
 SG       1243.0  26.571199  3.972171  19.0  23.0  26.0  30.00  38.0  1243.0   

                    ...      Year            eFG%                             \
              mean  ...       75%     max   count      mean       std    min   
Position            ...                                                        
 C        0.487674  ...    2013.0  2017.0  1370.0  0.492394  0.085248  0.000   
 F        0.450762  ...    2015.0  2017.0   324.0  0.476275  0.089104  0.000   
 G        0.403738  ...    2015.0  2017.0   351.0  0.461481  0.100343  0.000   
 GF       0.387000  ...    2006.0  2006.0     1.0  0.419000       NaN  0.419   
 PF       0.457102  ...    2013.0  2017.0  1359.0  0.479063  0.074629  0.000   
 PG       0.410105  ...    2014.0  2017.0  1309.0  0.459817  0.069349  0.000   
 SF       0.424452  ...    2013.0  2017.0  1136.0  0.474299  0.075337  0.000   
 SG       0.417027  ...    2014.0  2017.0  1243.0  0.477006  0.079841  0.000   

                                          
             25%     50%      75%    max  
Position                                  
 C        0.4490  0.4970  0.53700  1.000  
 F        0.4465  0.4815  0.51525  1.000  
 G        0.4290  0.4710  0.50350  1.500  
 GF       0.4190  0.4190  0.41900  0.419  
 PF       0.4470  0.4850  0.51800  1.000  
 PG       0.4300  0.4690  0.50000  0.679  
 SF       0.4470  0.4840  0.51400  0.900  
 SG       0.4495  0.4840  0.51500  1.500  

[8 rows x 72 columns]

In [159]:
# ditch wrong data
player_normalized["Position"] = player_normalized["Position"].str.strip()
player_normalized = player_normalized[player_normalized["Position"] != "GF"]
player_normalized.head()

# Change all ambivalent positions to its less valuable possibility because we don't have time for their feelings
# Basically, all Fs will be Power Forwards, and all Gs will be Shooting Guards
# Sorry not sorry
player_normalized["Position"] = player_normalized["Position"].replace("F", "PF").replace("G", "SG")

In [160]:
# analyze position info again
# The five basketball positions normally employed by organized basketball teams are 
# the point guard (PG), the shooting guard (SG), the small forward (SF), the power forward (PF), and the center (C).
player_gby = player_normalized.groupby("Position")
player_gby.describe()

Age                                                        FG%  \
           count       mean       std   min   25%   50%   75%   max   count   
Position                                                                      
C         1370.0  26.608759  3.981357  18.0  24.0  26.0  30.0  40.0  1370.0   
PF        1683.0  27.076055  4.190206  19.0  24.0  27.0  30.0  40.0  1683.0   
PG        1309.0  27.000000  4.238494  19.0  24.0  26.0  30.0  39.0  1309.0   
SF        1136.0  26.433979  4.091363  18.0  23.0  26.0  29.0  40.0  1136.0   
SG        1594.0  26.698871  4.163894  19.0  23.0  26.0  30.0  39.0  1594.0   

                    ...      Year            eFG%                           \
              mean  ...       75%     max   count      mean       std  min   
Position            ...                                                      
C         0.487674  ...    2013.0  2017.0  1370.0  0.492394  0.085248  0.0   
PF        0.455882  ...    2014.0  2017.0  1683.0  0.478526  0.077605  0.0   
PG        0.410105  ...    2014.0  2017.0  1309.0  0.459817  0.069349  0.0   
SF        0.424452  ...    2013.0  2017.0  1136.0  0.474299  0.075337  0.0   
SG        0.414100  ...    2014.0  2017.0  1594.0  0.473587  0.084992  0.0   

                                        
              25%    50%    75%    max  
Position                                
C         0.44900  0.497  0.537  1.000  
PF        0.44700  0.484  0.517  1.000  
PG        0.43000  0.469  0.500  0.679  
SF        0.44700  0.484  0.514  0.900  
SG        0.44525  0.481  0.513  1.500  

[5 rows x 72 columns]

In [161]:
# First plot: salary vs. position
player_gby.mean()

,Year,Age,FG%,eFG%,FT%,PTS,Salary,Salary Normalized,Salary Cap
Position,,,,,,,,,
C,2009.240146,26.608759,0.487674,0.492394,0.649775,430.786131,5.038984e+06,9.044884e+06,5.601907e+07
PF,2009.257873,27.076055,0.455882,0.478526,0.694858,491.915627,4.723975e+06,8.799407e+06,5.604386e+07
PG,2009.790680,27.000000,0.410105,0.459817,0.773484,544.163484,4.186819e+06,7.439247e+06,5.717689e+07
SF,2009.679577,26.433979,0.424452,0.474299,0.744118,565.804577,4.518945e+06,8.005047e+06,5.680899e+07
SG,2009.981179,26.698871,0.414100,0.473587,0.768490,546.489335,4.005127e+06,7.085374e+06,5.804027e+07
